In [1]:
!pip install ultralytics --upgrade -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
data_paths = {
    "Calculus": "/kaggle/input/oral-diseases/Calculus/Calculus",
    "Caries": "/kaggle/input/oral-diseases/Data caries/Data caries/caries orignal data set/done",
    "Gingivitis": "/kaggle/input/oral-diseases/Gingivitis/Gingivitis",
    "Ulcer": "/kaggle/input/oral-diseases/Mouth Ulcer/Mouth Ulcer/ulcer original dataset/ulcer original dataset",
    "ToothDiscoloration": "/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /tooth discoloration original dataset/tooth discoloration original dataset",
    "Hypodontia": "/kaggle/input/oral-diseases/hypodontia/hypodontia"
}


In [4]:
base_yolo_dir = "/kaggle/working/dental_yolo"
images_dir = os.path.join(base_yolo_dir, "images")
labels_dir = os.path.join(base_yolo_dir, "labels")

In [5]:
for split in ["train", "val"]:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, split), exist_ok=True)

In [6]:
class_map = {cls_name: idx for idx, cls_name in enumerate(data_paths.keys())}


In [7]:
all_images = []
for cls, path in data_paths.items():
    for img_file in os.listdir(path):
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            all_images.append((os.path.join(path, img_file), cls))

In [8]:
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)


In [9]:
def save_yolo_format(image_list, split):
    for img_path, cls in image_list:
        filename = os.path.basename(img_path)
        name, _ = os.path.splitext(filename)

        dst_img_path = os.path.join(images_dir, split, filename)
        shutil.copy(img_path, dst_img_path)

        label_path = os.path.join(labels_dir, split, name + ".txt")
        class_id = class_map[cls]
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n") 

save_yolo_format(train_imgs, "train")
save_yolo_format(val_imgs, "val")

In [10]:
class_names = list(data_paths.keys()) 
data_yaml = f"""
path: {base_yolo_dir}
train: images/train
val: images/val
nc: {len(class_names)}
names: {class_names}
"""

with open(os.path.join(base_yolo_dir, "data.yaml"), "w") as f:
    f.write(data_yaml)

In [12]:
from ultralytics import YOLO

model = YOLO('yolov10l.pt')


In [13]:
model.train(data=os.path.join(base_yolo_dir, "data.yaml"), epochs=300, imgsz=224, batch=16)

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dental_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1    137728  ultralytics.nn.modules.block.SCDown          [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1    268800  ultralytics

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 950.1±520.9 MB/s, size: 27.7 KB)


train: Scanning /kaggle/working/dental_yolo/labels/train... 3089 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3089/3089 [00:01<00:00, 1583.16it/s]

train: New cache created: /kaggle/working/dental_yolo/labels/train.cache


...

                   all       1012       1012      0.701      0.671      0.707      0.688
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 98, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



198 epochs completed in 2.521 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv10l summary (fused): 174 layers, 24,313,954 parameters, 0 gradients, 120.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  6.87it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1012       1012      0.674      0.715      0.744      0.733
              Calculus        243        243      0.482      0.765      0.627      0.627
                Caries         37         37      0.731      0.649      0.785      0.785
            Gingivitis        441        441      0.641        0.9      0.826      0.826
                 Ulcer         39         39      0.898      0.906      0.976      0.915
    ToothDiscoloration         26         26      0.672      0.462      0.599      0.599
            Hypodontia        226        226      0.618      0.611      0.647      0.647
Speed: 0.0ms preprocess, 2.2ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ede1c71bf50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [14]:
model.val()


Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv10l summary (fused): 174 layers, 24,313,954 parameters, 0 gradients, 120.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 544.4±388.5 MB/s, size: 19.3 KB)


val: Scanning /kaggle/working/dental_yolo/labels/val.cache... 1012 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1012/1012 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:05<00:00, 12.16it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1012       1012      0.674      0.716      0.742      0.732
              Calculus        243        243      0.482      0.765      0.626      0.626
                Caries         37         37      0.731      0.649      0.785      0.785
            Gingivitis        441        441      0.641      0.901      0.826      0.826
                 Ulcer         39         39      0.898      0.907      0.976      0.915
    ToothDiscoloration         26         26       0.67      0.462      0.593      0.593
            Hypodontia        226        226      0.619       0.61      0.647      0.647
Speed: 0.1ms preprocess, 2.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ede812ec5d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     